In [1]:
import pandas as pd
import numpy as np
import fileinput
from IPython.display import display
import ipywidgets as widgets
import PyPDF2
import io

In [30]:
extracted_text = ""

# def handle_pdf_upload(change):
#     print ('Start')
#     global extracted_text
    
#     uploaded_file = list(change.new.values())[0]
#     content = uploaded_file['content']
    
#     reader = PyPDF2.PdfFileReader(io.BytesIO(content))
#     text = ''
#     for page in reader.pages:
#         text += page.extract_text()

#     # Further processing with the extracted text
#     extracted_text = text
#     print(text)
#     print('PDF submitted successfully!')

# file_selector = widgets.FileUpload(accept='.xlsx', multiple = False)
# file_selector.observe(handle_pdf_upload, names='value')

def handle_file_upload(change):
    print('Start')
    global extracted_text

    uploaded_file = list(change.new.values())[0]
    content = uploaded_file['content']
    file_type = uploaded_file['metadata']['name'].split('.')[-1]  # Extract file type from name

    if file_type == 'xlsx':
        data = pd.read_excel(io.BytesIO(content))
    elif file_type == 'csv':
        data = pd.read_csv(io.StringIO(content.decode('utf-8')))  # Decoding needed for CSV file
    
    # Converting DataFrame to string
    text = data.to_string(index = False)

    # Further processing with the extracted text
    extracted_text = text
    print(text)
    print(f'{file_type.upper()} file submitted successfully!')

file_selector = widgets.FileUpload(accept='.xlsx, .csv', multiple = False)
file_selector.observe(handle_file_upload, names='value')

display(file_selector)

FileUpload(value={}, accept='.xlsx, .csv', description='Upload')

Start
                             Cover - shares                         3 Months Ended    Unnamed: 2
                                        NaN                          Mar. 31, 2023 Apr. 13, 2023
          Document Information [Line Items]                                                     
                              Document Type                                   10-Q              
                  Document Quarterly Report                                   true              
                   Document Period End Date                         Mar. 31,  2023              
                 Document Transition Report                                  false              
                         Entity File Number                              001-39039              
                     Entity Registrant Name                       Cloudflare, Inc.              
Entity Incorporation, State or Country Code                                     DE              
           Entity Tax Id

In [10]:
extracted_text

' Corrected Transcript  \n \n \n1-877-FACTSET   www.callstreet.com  Total Pages : 18 \nCopyright © 2001 -2023  FactSet  CallStreet, LLC  \n \n27-Apr-2023  \nCloudflare, Inc.  (NET ) \nQ1 2023 Earnings Call    Cloudflare, Inc.  (NET) \nQ1 2023 Earnings Call  Corrected Transcript  \n27-Apr-2023   \n \n1-877-FACTSET   www.callstreet.com   2 \nCopyright © 2001 -2023  FactSet  CallStreet, LLC  \n \n \nCORPORATE PARTICIPANTS  \n \nPhilip Winslow  \nVice President -Strategic Finance & Treasury, Cloudflare, Inc.  \nMatthew Prince  \nChairman, Co -Founder & Chief Executive Officer, Cloudflare, Inc.  Thomas Josef Seifert  \nChief Financial Officer, Cloudflare, Inc.  \n ................................ ................................ ................................ ................................ ................................ ................................ ................................ ................................ .....  \n \nOTHER PARTICIPANTS  \n \nSterling Auty  \nAnalyst, Moffe

In [3]:
import openai

# Load API key from a separate file
with open("openai.txt", "r") as file:
    api_key = file.read().strip()

openai.api_key = api_key

In [27]:
import textwrap

generated_text = ""

def generate_text_with_openai(extracted_text):
    chunk_size=12000 # adjust based on the used model
    chunks = textwrap.wrap(extracted_text, chunk_size)

    generated_texts = []

    for chunk in chunks:

#         # Set the prompt for the OpenAI model
#         prompt = f"You are a financial analysis with 20 years of experience. You are set to analyze, \
#                     summarize, identify and highlight information from conference call transcript. \
#                     Here is the transscript: {extracted_text}"

        # Set the desired number of tokens for the generated text
        max_tokens = 12000  # Adjust the value as per your requirement

        # Generate text using the OpenAI model
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k", # change model here depends on documentation
#             prompt=prompt,
            messages=[
                {
                    "role": "system",
                    "content": "You are a financial analysis with 20 years of experience. \
                                User will give you transcript, you will analyze, summarize, identify and highlight \
                                information from conference call transcript. \
                                You will firstly assess if this call is positive, neutral or negative for the company.\
                                The other information is in bullet point"
                },
                {
                    "role": "user",
                    "content": chunk
                }
            ],
            max_tokens=max_tokens,
            n=1,
            stop=None,
            temperature=0.7
        )

        # Extract the generated text from the OpenAI response
        generated_text = response.choices[0].message.content.strip()
        
        generated_texts.append(generated_text)
        
    # Combine all the generated text chunks into one string
    full_generated_text = " ".join(generated_texts)

    return full_generated_text

generated_text = generate_text_with_openai(extracted_text)

In [28]:
generated_text

"- The Q1 2023 earnings call for Cloudflare, Inc. took place on April 27, 2023.\n- The call was led by Philip Winslow, Vice President of Strategic Finance, Treasury, and Investor Relations.\n- Other participants on the call included analysts from MoffettNathanson LLC, KeyBanc Capital Markets, Jefferies LLC, RBC Capital Markets LLC, Morgan Stanley & Co. LLC, Piper Sandler & Co., Truist Securities, Inc., and JPMorgan Securities LLC.\n- Matthew Prince, Chairman, Co-Founder, and Chief Executive Officer of Cloudflare, Inc., also spoke during the call.\n- Q1 was a quarter of contrast for Cloudflare. They achieved revenue of $290.2 million, up 37% YoY, and their gross margin was 77.8%.\n- They added 114 new large customers in the quarter, bringing the total to 2,156, a YoY increase of 40%.\n- Their dollar-based net retention fell to 117%, down 5% QoQ, but they did not see elevated churn across their customer base.\n- Cloudflare experienced tailwinds and headwinds during the quarter. Their new